# Обучение моделей временных рядов (Prophet)

**Цель:** Создать и сохранить две специализированные модели Prophet для нашего AI-агента.

## Введение: Почему Prophet?

Статические пороги (`if response_time > 500ms`) неэффективны, поскольку они не учитывают естественные циклы нагрузки системы (день/ночь, будни/выходные). Это приводит к ложным тревогам в пиковые часы и пропущенным аномалиям в периоды затишья.

**Prophet** — это библиотека от Facebook для прогнозирования временных рядов, которая автоматически моделирует эти сезонные компоненты. Она позволяет нам создать динамический "коридор нормы", специфичный для каждого момента времени.

Мы создадим две модели:
1.  **`anomaly_detector.pkl`**: Упрощенная, сверхчувствительная модель для надежного обнаружения резких выбросов в прошлом.
2.  **`failure_predictor.pkl`**: Более сложная модель с учетом сезонности для точного прогнозирования будущих сбоев.


## Шаг 1: Подготовка и генерация данных

В реальной MLOps-системе на этом шаге мы бы подключились к продакшен-базе данных и выгрузили метрики за последний месяц. Для демонстрации мы сгенерируем реалистичный набор данных.

In [1]:
import pandas as pd
import pickle
from prophet import Prophet
import os
import warnings

warnings.filterwarnings('ignore', category=FutureWarning)

# Определяем директорию для сохранения готовых моделей
MODELS_DIR = "trained_models"
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)
    print(f"Директория '{MODELS_DIR}' создана.")
else:
    print(f"Директория '{MODELS_DIR}' уже существует.")

C:\Users\kosty\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Директория 'trained_models' уже существует.


In [2]:
print("Генерация исторических данных для Prophet...")
# Симулируем 30 дней данных с поминутной детализацией
history_dates = pd.to_datetime(pd.date_range('2025-08-20', periods=1440 * 30, freq='T'))
# Симулируем суточные и недельные циклы
history_metrics = [150 + i % 144 * 1.5 + (i // (1440*7)) * 10 for i in range(len(history_dates))]
metrics_df = pd.DataFrame({'timestamp': history_dates, 'response_time_ms': history_metrics})

# Prophet требует специфические названия колонок: 'ds' для даты и 'y' для значения
prophet_df = metrics_df[['timestamp', 'response_time_ms']].rename(
    columns={'timestamp': 'ds', 'response_time_ms': 'y'}
)

print("Обучающие данные для Prophet готовы. Размер: ", prophet_df.shape)
prophet_df.tail()

Генерация исторических данных для Prophet...
Обучающие данные для Prophet готовы. Размер:  (43200, 2)


,ds,y
43195,2025-09-18 23:55:00,398.5
43196,2025-09-18 23:56:00,400.0
43197,2025-09-18 23:57:00,401.5
43198,2025-09-18 23:58:00,403.0
43199,2025-09-18 23:59:00,404.5


## Шаг 2: Обучение модели для детекции аномалий (`anomaly_detector.pkl`)

Эта модель используется для **реактивного** анализа. Ее задача — с максимальной надежностью находить резкие выбросы в прошлом. Для этого мы убираем все сложные сезонные компоненты, которые могут "смазать" картину на коротких интервалах, и заставляем модель строить простую линейную регрессию.

In [3]:
print("Обучение модели 'anomaly_detector.pkl'...")
model_anomalies = Prophet(
    interval_width=0.99,          # Очень широкий доверительный интервал
    growth='linear',              # Простой линейный рост
    daily_seasonality=False,      # Отключаем все, что может внести шум в тестах
    weekly_seasonality=False,
    yearly_seasonality=False,
    changepoints=None             # Запрещаем искать точки излома тренда
)
model_anomalies.fit(prophet_df)

# Сериализация с использованием стандартной библиотеки pickle
model_path = os.path.join(MODELS_DIR, "anomaly_detector.pkl")
with open(model_path, 'wb') as f:
    pickle.dump(model_anomalies, f)

print(f" -> Модель для детекции аномалий успешно сохранена в '{model_path}'")

Обучение модели 'anomaly_detector.pkl'...


01:19:42 - cmdstanpy - INFO - Chain [1] start processing
01:19:51 - cmdstanpy - INFO - Chain [1] done processing


 -> Модель для детекции аномалий успешно сохранена в 'trained_models\anomaly_detector.pkl'


## Шаг 3: Обучение модели для прогнозирования сбоев (`failure_predictor.pkl`)

Эта модель используется для **проактивного** анализа. Ее задача — заглянуть в будущее. Для этого, наоборот, мы включаем все сезонные компоненты, чтобы модель могла построить максимально точный и реалистичный прогноз.

In [4]:
print("Обучение модели 'failure_predictor.pkl'...")
model_potential_failures = Prophet(
    interval_width=0.95,          # Стандартный доверительный интервал
    daily_seasonality=True,       # Включаем учет суточных циклов
    weekly_seasonality=True,      # Включаем учет недельных циклов
    changepoint_prior_scale=0.05  # Стандартная чувствительность к изменениям тренда
)
model_potential_failures.fit(prophet_df)

# Сериализация с использованием pickle
model_path = os.path.join(MODELS_DIR, "failure_predictor.pkl")
with open(model_path, 'wb') as f:
    pickle.dump(model_potential_failures, f)

print(f" -> Модель для прогнозирования сбоев успешно сохранена в '{model_path}'")

Обучение модели 'failure_predictor.pkl'...


01:19:53 - cmdstanpy - INFO - Chain [1] start processing
01:20:03 - cmdstanpy - INFO - Chain [1] done processing


 -> Модель для прогнозирования сбоев успешно сохранена в 'trained_models\failure_predictor.pkl'


## Заключение

УРАА!